## 針對 Single Error Correction(SEC)做 Synthesizer
### python(.py) to verilog(.v)

#### Define function: (Not including 2's complement range)

In [1]:
import math
import numpy as np


def decimal_to_binary(decimal_number):
    if decimal_number == 0:
        return "0"
    # 處理正數：直接轉換
    if decimal_number > 0:
        return bin(decimal_number)[2:]  # 移除 "0b" 前綴
       # 處理負數：自動計算所需位元數（2's complement）
    abs_value = abs(decimal_number)
    bits = abs_value.bit_length() + 1  # 負數多一個符號位元
    two_complement_value = (1 << bits) + decimal_number
    
    return bin(two_complement_value)[2:]  # 移除 "0b" 前綴

def bits_needed(number):
    return number.bit_length()


def bits_range(bits, signed=False):     ## 可選擇設定 有號數 或 無號數
    if bits <= 0:
        return "位元數必須為正整數！"
    
    if signed:
        # 二補數範圍: -2^(bits-1) 到 2^(bits-1)-1
        min_value = -(2**(bits - 1))
        max_value = 2**(bits - 1) - 1
    else:
        # 無符號範圍: 0 到 2^bits - 1
        min_value = 0
        max_value = 2**bits - 1

    return min_value, max_value


print("一個2補數所需要的位元數",bits_needed(120))
print("一bits數可表示的2補數的範圍",bits_range(bits_needed(120)))
print("一個2補數表達為二進制為:",decimal_to_binary(-8))

一個2補數所需要的位元數 7
一bits數可表示的2補數的範圍 (0, 127)
一個2補數表達為二進制為: 11000


### 選取特定的位元數:

In [2]:
### 輸入數值為data, ANcode的A為AN
### 乘完A後的數值為 input
data = 1073741823   ## 
A = 18613
input = data*A
bits = bits_needed(input)
print("AN資料的位元數:", bits)
print(bits_needed(A))
print(bits_needed(data))

AN資料的位元數: 45
15
30


### 產生1-bit AWE (Arithmetic-Weight Error)

In [3]:
AWE_1=[]
e1=0     ## 錯誤數量
e1_data=[]
for i in range(0, bits):
    e1_data.insert(e1 , input + 2**i)
    AWE_1.insert(e1, 2**i)
    e1 = e1 + 1
    e1_data.insert(e1 , input - 2**i)
    AWE_1.insert(e1, -2**i)
    e1 = e1 + 1

print(bits)
print(input)
print("產生1-bit AWE:",AWE_1)
print("算術錯誤的資料:", e1_data)
print(e1)
print(np.size(AWE_1))

45
19985556551499
產生1-bit AWE: [1, -1, 2, -2, 4, -4, 8, -8, 16, -16, 32, -32, 64, -64, 128, -128, 256, -256, 512, -512, 1024, -1024, 2048, -2048, 4096, -4096, 8192, -8192, 16384, -16384, 32768, -32768, 65536, -65536, 131072, -131072, 262144, -262144, 524288, -524288, 1048576, -1048576, 2097152, -2097152, 4194304, -4194304, 8388608, -8388608, 16777216, -16777216, 33554432, -33554432, 67108864, -67108864, 134217728, -134217728, 268435456, -268435456, 536870912, -536870912, 1073741824, -1073741824, 2147483648, -2147483648, 4294967296, -4294967296, 8589934592, -8589934592, 17179869184, -17179869184, 34359738368, -34359738368, 68719476736, -68719476736, 137438953472, -137438953472, 274877906944, -274877906944, 549755813888, -549755813888, 1099511627776, -1099511627776, 2199023255552, -2199023255552, 4398046511104, -4398046511104, 8796093022208, -8796093022208, 17592186044416, -17592186044416]
算術錯誤的資料: [19985556551500, 19985556551498, 19985556551501, 19985556551497, 19985556551503, 199855565

In [59]:
# 計算對應元素相減
result = [a - b for a, b in zip(e1_data, AWE_1)]
# 顯示結果
print(result)
print(set(result))

[19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 19985556551499, 1998555

### 產生SEC的模數:

In [49]:
def custom_mod(a, b):
    if b == 0:
        return "錯誤：除數不能為零！"
    # 計算商和餘數，修正餘數計算邏輯
    quotient = a // b
    remainder = a - b * quotient
    # 修正負餘數的情況
    if remainder > 0 and a < 0:
        remainder -= abs(b)
        quotient += 1
    return remainder

SEC_R = []
for m in range(0,e1):
    mod = custom_mod(e1_data[m], A)
    SEC_R.append(mod)
    
sec_r = []
for m in range(0,e1):
    mod = e1_data[m] % A
    sec_r.append(mod)

print("SEC的模數:\n",SEC_R)
print("兩種取餘數法是否相同:",sec_r == SEC_R)
print("產生1-bit AWE:\n",AWE_1)
print("SEC的大小:",np.size(SEC_R))
print("1-bit AWE 大小:",np.size(AWE_1))

SEC的模數:
 [1, 18612, 2, 18611, 4, 18609, 8, 18605, 16, 18597, 32, 18581, 64, 18549, 128, 18485, 256, 18357, 512, 18101, 1024, 17589, 2048, 16565, 4096, 14517, 8192, 10421, 16384, 2229, 14155, 4458, 9697, 8916, 781, 17832, 1562, 17051, 3124, 15489, 6248, 12365, 12496, 6117, 6379, 12234, 12758, 5855, 6903, 11710, 13806, 4807, 8999, 9614, 17998, 615, 17383, 1230, 16153, 2460, 13693, 4920, 8773, 9840, 17546, 1067, 16479, 2134, 14345, 4268, 10077, 8536, 1541, 17072, 3082, 15531, 6164, 12449, 12328, 6285, 6043, 12570, 12086, 6527, 5559, 13054, 11118, 7495, 3623, 14990]
兩種取餘數法是否相同: True
產生1-bit AWE:
 [1, -1, 2, -2, 4, -4, 8, -8, 16, -16, 32, -32, 64, -64, 128, -128, 256, -256, 512, -512, 1024, -1024, 2048, -2048, 4096, -4096, 8192, -8192, 16384, -16384, 32768, -32768, 65536, -65536, 131072, -131072, 262144, -262144, 524288, -524288, 1048576, -1048576, 2097152, -2097152, 4194304, -4194304, 8388608, -8388608, 16777216, -16777216, 33554432, -33554432, 67108864, -67108864, 134217728, -134217728, 2

### 將數值轉換為2補數:

In [50]:
def to_twos_complement(n, bits):
    """
    將整數 n 轉換為指定位元數 bits 的二補數二進位字串
    """
    if n < 0:
        # 負數以二補數表示：加上 2^bits
        n = (1 << bits) + n
    # 格式化為 bits 位元的二進位字串，不足位數補 0
    return format(n, '0{}b'.format(bits))


twos_complement_AWE =[to_twos_complement(AWE, bits=bits+1) for AWE in AWE_1]
print(twos_complement_AWE)
print(np.size(twos_complement_AWE))


['0000000000000000000000000000000000000000000001', '1111111111111111111111111111111111111111111111', '0000000000000000000000000000000000000000000010', '1111111111111111111111111111111111111111111110', '0000000000000000000000000000000000000000000100', '1111111111111111111111111111111111111111111100', '0000000000000000000000000000000000000000001000', '1111111111111111111111111111111111111111111000', '0000000000000000000000000000000000000000010000', '1111111111111111111111111111111111111111110000', '0000000000000000000000000000000000000000100000', '1111111111111111111111111111111111111111100000', '0000000000000000000000000000000000000001000000', '1111111111111111111111111111111111111111000000', '0000000000000000000000000000000000000010000000', '1111111111111111111111111111111111111110000000', '0000000000000000000000000000000000000100000000', '1111111111111111111111111111111111111100000000', '0000000000000000000000000000000000001000000000', '1111111111111111111111111111111111111000000000',

In [51]:
#### SEC_AN Decoder
# 2025/2/10
#-------------------------------------
# input:e1_data, output: corrected value
Bw = bits
Bn = bits_needed(data)
BA = bits_needed(A)
#-------------------------------------
fn = 'SEC_LUT_Decoder' + str(bits_needed(data)) + "bits" +'.v'
f = open(fn, 'w')

f.write('// Product (AN) Code SEC_LUT_Decoder\n')
f.write('// ' + fn + '\n')
f.write('// Received codeword W = AN + e, e is single arithmetic weight error (AWE), +2^i or -2^i.\n')
f.write('module SEC_LUT_Decoder%dbits(W, N);\n'% (bits_needed(data)))
f.write('input \t[%d:0]\tW;\n' % (Bw-1))
f.write('output\t[%d:0]\tN;\n' % (Bn-1))
f.write('parameter A = %d;\n\n' % (A))
f.write('wire \t[%d:0]\tQ;\n' % (Bn-1))
f.write('wire \t[%d:0]\tR;\n' % (BA-1))
f.write('assign Q = W / A;\n')
f.write('assign R = W - (A * Q);\n\n')

f.write('reg\tsigned\t[%d:0]\tDelta;\n' % (Bw))
f.write('always@(*) begin\n')
f.write('\tcase(R)\n')
for i in range(2*Bw):
    f.write("\t\t%d: Delta = %d'sb%s;\n" % (SEC_R[i], Bw+1 ,twos_complement_AWE[i]))
f.write("\t\tdefault: Delta =%d'sb0;\n" % (Bw+1))
f.write('\tendcase\n')
f.write('end\n\n')
f.write('assign N = (W - Delta) / A;\n\n')

f.write('endmodule\n')
f.close();
print('Module file %s is generated.\n' % (fn))

Module file SEC_LUT_Decoder30bits.v is generated.



In [20]:
print(custom_mod(-10,3))
print(-10%3)

-1
2
